<a href="https://colab.research.google.com/github/Pragadesh-45/AI-Notebooks/blob/main/Model_Explainability_using_SHAP_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
!git clone https://github.com/yuquandu/Data-driven-Ship-Fuel-Efficiency-Modeling
%cd /content/Data-driven-Ship-Fuel-Efficiency-Modeling
!unzip TrainedModels.zip
!pip install shap


Cloning into 'Data-driven-Ship-Fuel-Efficiency-Modeling'...
remote: Enumerating objects: 77, done.
remote: Counting objects: 100% (77/77), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 77 (delta 19), reused 61 (delta 11), pack-reused 0
Receiving objects: 100% (77/77), 3.24 MiB | 25.35 MiB/s, done.
Resolving deltas: 100% (19/19), done.
Error downloading object: TrainedModels.zip (d395556): Smudge error: Error downloading TrainedModels.zip (d3955560fdedbb37eb48d812efd88220cb30740c63f307ae0959bdb6cd59c95d): batch response: This repository is over its data quota. Account responsible for LFS bandwidth should purchase more data packs to restore access.

Errors logged to /content/Data-driven-Ship-Fuel-Efficiency-Modeling/Data-driven-Ship-Fuel-Efficiency-Modeling/.git/lfs/logs/20231219T182848.024339531.log
Use `git lfs logs last` to view the log.
error: external filter 'git-lfs filter-process' failed
fatal: TrainedModels.zip: smudge filter lfs failed
You can inspect what

In [20]:
# Import machine learning models packages in Python
import os
import time
import joblib
import openpyxl
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn import svm
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import learning_curve
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, explained_variance_score

import xgboost as xgb
from xgboost.sklearn import XGBRegressor
import shap

import requests
from io import BytesIO
import random

In [22]:
# Load model
model_url = "https://raw.githubusercontent.com/Pragadesh-45/maritime/main/Ship_S5_XG_Sensor2"
response = requests.get(model_url)
model_ = joblib.load(BytesIO(response.content))

path_model = model_


[18:31:09] WARNING: /workspace/src/learner.cc:873: Found JSON model saved before XGBoost 1.6, please save the model using current version again. The support for old JSON model will be discontinued in XGBoost 2.3.


In [53]:
import pandas as pd

# # Lowest Values
# sailingSpeed = 12
# draft = 7
# trim = -2.0
# windSpeed = 0
# windDirection = 0
# currentSpeed = 0
# currentDirection = 0
# combinedWaveHeight = 0
# combinedWaveDirection = 0
# combinedWavePeriod = 0
# seaWaterTemp = 0

#Highest Values
# sailingSpeed = 26
# draft = 14
# trim = 5
# windSpeed = 40
# windDirection = 180
# currentSpeed = 3.5
# currentDirection = 180
# combinedWaveHeight = 8
# combinedWaveDirection = 180
# combinedWavePeriod = 15
# seaWaterTemp = 35

#Highest Values

# Create a list of values
values = [sailingSpeed, draft, trim, windSpeed, windDirection, currentSpeed,
          currentDirection, combinedWaveHeight, combinedWavePeriod,
          combinedWaveDirection, seaWaterTemp]

# Create a DataFrame with a single row and 11 columns
x_expr = pd.DataFrame([values], columns=['sailingSpeed', 'draft', 'trim', 'windSpeed', 'windDirection',
                                          'currentSpeed', 'currentDirection', 'combinedWaveHeight',
                                          'combinedWavePeriod', 'combinedWaveDirection', 'seaWaterTemp'])


In [54]:
x_expr

,sailingSpeed,draft,trim,windSpeed,windDirection,currentSpeed,currentDirection,combinedWaveHeight,combinedWavePeriod,combinedWaveDirection,seaWaterTemp
0,17.819268,9.523762,3.033185,28.739523,150.915178,0.221266,164.832166,4.459258,4.609775,38.30329,34.395791


In [55]:
import random
# Sailing speed (knots); Min value = 12; Maximum value = 26;
sailingSpeed = random.uniform(12, 26)
# Draft (m); Min value = 7; Maximum value = 14;
draft = random.uniform(7, 14)
# Trim (m); Min value = -2.0; Maximum value = 5;
trim = random.uniform(-2.0, 5)
# Wind speed (knots); Min value = 0 ; Maximum value = 40;
windSpeed = random.uniform(0, 40)
# Wind direction (Rel.) (degree); Min value = 0; Maximum value = 180;
windDirection = random.uniform(0, 180)
# Sea current speed (knots); Min value = 0; Maximum value = 3.5;
currentSpeed = random.uniform(0, 3.5)
# Sea current direction (Rel.) (degree); Min value = 0; Maximum value = 180;
currentDirection = random.uniform(0, 180)
# Combined wave height (m); Min value = 0; Maximum value = 8;
combinedWaveHeight = random.uniform(0, 8)
# Combined wave direction (Rel.) (degree); Min value = 0; Maximum value = 180;
combinedWaveDirection = random.uniform(0, 180)
# Combined wave period; Min value = 0; Maximum value = 15
combinedWavePeriod = random.uniform(0, 15)
# Sea water temperature (degree); Min value = 0; Maximum value = 35;
seaWaterTemp = random.uniform(0, 35)

# x_expr = pd.DataFrame([sailingSpeed, draft, trim, windSpeed, windDirection, currentSpeed, currentDirection,
#                        combinedWaveHeight, combinedWavePeriod, combinedWaveDirection, seaWaterTemp],
#                       columns = ['sailingSpeed', 'draft', 'trim', 'windSpeed', 'windDirection', 'currentSpeed',
#                                  'currentDirection', 'combinedWaveHeight', 'combinedWavePeriod',
#                                  'combinedWaveDirection', 'seaWaterTemp'])

In [56]:
x_expr_train = x_expr.values.reshape(1,-1)
print(x_expr.shape)

# load the trained machine learning model
model = model_

# Predict the y (fuel consumption rate, MT/day) values of the experimental data "x_expr"
y_pre = model.predict(x_expr_train)

print("The prediction of ship fuel consumption rate of machine learning model is (metric ton/day):", y_pre* 24 / 1000)

(1, 11)
The prediction of ship fuel consumption rate of machine learning model is (metric ton/day): [96.61908]


In [57]:
explainer = shap.Explainer(model)
shap.explainers.Tree(model, data=x_expr, model_output='raw')


[18:43:53] WARNING: /workspace/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
[18:43:58] WARNING: /workspace/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
[18:44:01] WARNING: /workspace/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.


In [58]:
# Calculate SHAP values on the test set
shap_values = explainer.shap_values(x_expr_train)

# Summary plot to visualize feature importance
# shap.summary_plot(shap_values, x_expr_train, feature_names=x_expr.columns, show=False)

# Find the most contributing feature
most_contributing_feature_index = np.argmax(np.abs(shap_values))
most_contributing_feature_name = x_expr.columns[most_contributing_feature_index]

In [62]:
predi = "The prediction of ship fuel consumption rate of machine learning model is (metric ton/day):" + str(y_pre* 24 / 1000)
most_contributing_feature_name = str(most_contributing_feature_name)
# print("The prediction of ship fuel consumption rate of machine learning model is (metric ton/day):", y_pre* 24 / 1000)
# print(most_contributing_feature_name)

x = predi + "\n" + most_contributing_feature_name

print(x)

The prediction of ship fuel consumption rate of machine learning model is (metric ton/day):[96.61908]
sailingSpeed
